# Cora Dataset

In [7]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='.',name='Cora')

Processing...
Done!


In [14]:
data = dataset[0] #Cora 데이터셋은 single graph dataset

In [13]:
dir(dataset) #객체가 갖고 있는 매서드 확인(=어떤 기능을 제공하는가?)

['__add__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_data',
 '_data_list',
 '_download',
 '_indices',
 '_infer_num_classes',
 '_is_protocol',
 '_process',
 'collate',
 'copy',
 'cpu',
 'cuda',
 'data',
 'download',
 'force_reload',
 'geom_gcn_url',
 'get',
 'get_summary',
 'has_download',
 'has_process',
 'index_select',
 'indices',
 'len',
 'load',
 'log',
 'name',
 'num_classes',
 'num_edge_features',
 'num_features',
 'num_node_features',
 'pre_filter',
 'pre_transform',
 'print_summary',
 'process',
 'proces

In [24]:
print(f'Dataset : {dataset}')
print('-----------')
print(f'Number of Graphs : {len(dataset)}')
print(f'Number of Nodes : {data.x.shape[0]}') #2708개의 노드
print(f'Number of features : {data.num_features}') #1433차원
print(f'Number of classes : {dataset.num_classes}') #클래스 갯수

Dataset : Cora()
-----------
Number of Graphs : 1
Number of Nodes : 2708
Number of features : 1433
Number of classes : 7


In [31]:
#data의 세부정보(그래프 데이터셋의 세부 정보)
print(f'edges are directed : {data.is_directed()}')
print(f'Graph has isolated nodes : {data.has_isolated_nodes()}')
print(f'Graph has self loop : {data.has_self_loops()}')

edges are directed : False
Graph has isolated nodes : False
Graph has self loop : False


In [42]:
import pandas as pd
import numpy as np
import torch

In [38]:
print(data.x.shape,data.y.shape) #노드와 레이블 형태

torch.Size([2708, 1433]) torch.Size([2708])


In [39]:
df_x = pd.DataFrame(data.x.numpy())

In [40]:
df_x['label'] = pd.DataFrame(data.y.numpy())

In [41]:
df_x

,0,1,2,3,4,5,6,7,8,9,...,1424,1425,1426,1427,1428,1429,1430,1431,1432,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2704,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2705,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
2706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [43]:
#평가 지표 함수
def accuracy(y_pred,y_true):
    return torch.sum(y_pred==y_true)/len(y_true)

In [45]:
from torch.nn import Linear
import torch.nn.functional as F

In [81]:
class MLP(torch.nn.Module):
    def __init__(self,dim_in,dim_h,dim_out):
        super().__init__()
        self.linear1 = Linear(dim_in,dim_h)
        self.linear2 = Linear(dim_h,dim_out)
    
    def forward(self,x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x #criterion에 이미 소프트맥스가 존재하므로, x그대로 반환
    
    def fit(self,data,epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),lr=0.01,weight_decay=5e-4)

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x)
            loss = criterion(out[data.train_mask],data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            
            if epoch%20==0:
                val_loss = criterion(out[data.val_mask],data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),data.y[data.val_mask])
                print(f'Epoch : {epoch:>3} | Train_loss : {loss:.3f}% | Train_acc : {acc*100:>5.2f}% | Val_loss : {val_loss:.2f}% | Val_acc : {val_acc:.2f}%')

    def test(self,data):
        self.eval()
        out = self(data.x)
        acc = accuracy(out[data.test_mask].argmax(dim=1),data.y[data.test_mask])
        return acc


In [78]:
mlp = MLP(dataset.num_features,16,dataset.num_classes)
print(mlp)

MLP(
  (linear1): Linear(in_features=1433, out_features=16, bias=True)
  (linear2): Linear(in_features=16, out_features=7, bias=True)
)


In [79]:
mlp.fit(data,epochs=100)

Epoch :   0 | Train_loss : 1.950% | Train_acc : 10.71% | Val_loss : 1.95% | Val_acc : 0.15%
Epoch :  20 | Train_loss : 0.112% | Train_acc : 100.00% | Val_loss : 1.38% | Val_acc : 0.53%
Epoch :  40 | Train_loss : 0.014% | Train_acc : 100.00% | Val_loss : 1.45% | Val_acc : 0.53%
Epoch :  60 | Train_loss : 0.007% | Train_acc : 100.00% | Val_loss : 1.44% | Val_acc : 0.52%
Epoch :  80 | Train_loss : 0.008% | Train_acc : 100.00% | Val_loss : 1.36% | Val_acc : 0.54%
Epoch : 100 | Train_loss : 0.009% | Train_acc : 100.00% | Val_loss : 1.34% | Val_acc : 0.54%


In [82]:
acc = mlp.test(data)
print(f'Test_acc : {acc*100:.2f}%')

Test_acc : 54.90%


## GNN 구현

In [107]:
from torch_geometric.utils import to_dense_adj

adjacency = to_dense_adj(data.edge_index)[0] # 인접행렬로 변환 / 배치를 제외하고 접근
adjacency += torch.eye(len(adjacency)) # n차 항등행렬
adjacency

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        [0., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 1.]])

In [ ]:
class GNNlayer(torch.nn.Module):
    def __init__(self,dim_in,dim_out):
        super().__init__()
        self.linear = Linear(dim_in,dim_out,bias=True) #bias
    
    def forward(self,x,adjacency):
        x = self.linear(x)
        x = torch.sparse.mm(adjacency,x)
        return x
    
class GNN(torch.nn.Module):
    def __init__(self,dim_in,dim_h,dim_out):
        super().__init__()
        self.gnn1 = GNNlayer(dim_in,dim_h) #레이어 객체
        self.gnn2 = GNNlayer(dim_h,dim_out) #레이어 객체
    
    def forward(self,x,adjacency):
        h = self.gnn1(x,adjacency) #GNNlayer forward 호출
        h = F.relu(h)
        h = self.gnn2(h,adjacency) #GNNlayer forward 호출
        return h
    
    def fit(self,data,epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),lr=0.01,weight_decay=5e-4)

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x,adjacency) #GNN 클래스의 forward 매서드 실행
            loss = criterion(out[data.train_mask],data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),data.y[data.train_mask])
            loss.backward()
            optimizer.step()
            
            if epoch%20==0:
                val_loss = criterion(out[data.val_mask],data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),data.y[data.val_mask])
                print(f'Epoch : {epoch:>3} | Train_loss : {loss:.3f}% | Train_acc : {acc*100:>5.2f}% | Val_loss : {val_loss:.2f}% | Val_acc : {val_acc:.2f}%')

    def test(self,data):
        self.eval()
        out = self(data.x,adjacency)
        acc = accuracy(out[data.test_mask].argmax(dim=1),data.y[data.test_mask])
        return acc
        

In [112]:
gnn = GNN(dataset.num_features,16,dataset.num_classes)
print(gnn)

GNN(
  (gnn1): GNNlayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (gnn2): GNNlayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [113]:
gnn.fit(data,epochs=100)

Epoch :   0 | Train_loss : 2.699% | Train_acc : 17.14% | Val_loss : 2.50% | Val_acc : 0.22%
Epoch :  20 | Train_loss : 0.225% | Train_acc : 95.71% | Val_loss : 1.35% | Val_acc : 0.70%
Epoch :  40 | Train_loss : 0.035% | Train_acc : 100.00% | Val_loss : 1.74% | Val_acc : 0.71%
Epoch :  60 | Train_loss : 0.011% | Train_acc : 100.00% | Val_loss : 2.14% | Val_acc : 0.70%
Epoch :  80 | Train_loss : 0.005% | Train_acc : 100.00% | Val_loss : 2.25% | Val_acc : 0.71%
Epoch : 100 | Train_loss : 0.003% | Train_acc : 100.00% | Val_loss : 2.30% | Val_acc : 0.72%


In [119]:
acc = gnn.test(data)
print(f'Test_acc : {acc*100:.5f}%')

Test_acc : 70.40000%
